In [3]:
import csv

annotation_list = ["Tweet Annotations - Feb Translated Annotations.tsv"]
annotation_list.append("Tweet Annotations - Mar Translated Annotations.tsv")
annotation_list.append("Tweet Annotations - Apr Translated Annotations.tsv")

In [4]:
# Transfer the contents of a tsv file to a list for easier access
def tsv_to_list(annotation_file):
    annotation_list = []
    annotation = open(annotation_file, encoding='utf-8')
    read_tsv = csv.reader(annotation, delimiter="\t") 
    for row in read_tsv:
        annotation_list.append(row)
    return annotation_list

In [5]:
# Get the label for each verse for one annotator
# N: -1, N+NU: -0.5, NU: 0, P+NU: 0.5, P: 1
def get_labels_for(annotation):
    tweets_and_labels = []
    row_num = 0
    for row in annotation:
        if row_num == 0:
            row_num += 1
            continue
        if row[15] != '': # don't use unsure
            row_num += 1
            continue
            #print("not using unsure row", row_num+1)
        if 'UNUSABLE' in row[16].upper() or 'NOT ENOUGH INFO' in row[16].upper():
            row_num += 1
            continue
            #print("unusable row", row_num+1)
        elif row[6].upper() == 'X' and (row[8].upper() != 'X' and row[7].upper() == 'X'): # don't use pos + neg mixed labels
            row_num += 1
            continue
            #print("not using pos+neg row", row_num+1)
        elif row[6].upper() == 'X' and (row[7].upper() != 'X' and row[8].upper() != 'X'): # only pos
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 1]
            tweets_and_labels.append(new_row)
        #elif row[6].upper() == 'X' and (row[8].upper() == 'X' and row[7].upper() != 'X'): # pos + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 3]
        #    tweets_and_labels.append(new_row)
        elif row[8].upper() == 'X' and (row[6].upper() != 'X' and row[7].upper() != 'X'): # only neu 
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 0]
            tweets_and_labels.append(new_row)
        elif row[7].upper() == 'X' and (row[8].upper() != 'X' and row[6].upper() != 'X'): # only neg
            new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], -1]
            tweets_and_labels.append(new_row)
        #elif row[8].upper() == 'X' and (row[7].upper() == 'X' and row[6].upper() != 'X'): # neg + neu
        #    new_row = [annotation[row_num][0], annotation[row_num][1], annotation[row_num][2], annotation[row_num][3], annotation[row_num][4], annotation[row_num][5], 2]
        #    tweets_and_labels.append(new_row) 
        #else:
        #    print("outlier row:", row_num+1)
        row_num += 1
    return tweets_and_labels

In [7]:
feb_list = tsv_to_list(annotation_list[0])
feb_labels = get_labels_for(feb_list)
mar_list = tsv_to_list(annotation_list[1])
mar_labels = get_labels_for(mar_list)
apr_list = tsv_to_list(annotation_list[2])
apr_labels = get_labels_for(apr_list)

all_labels = feb_labels + mar_labels + apr_labels
#print(all_labels.count(-1), all_labels.count(0), all_labels.count(1))

import pandas as pd
df = pd.DataFrame(all_labels, columns=['tweetID', 'tweetTime', 'tweetText', 'tweetLang', 'tweetCoordinates', 'tweetPlace', 'tweetSentiment'])
df.head()


,tweetID,tweetTime,tweetText,tweetLang,tweetCoordinates,tweetPlace,tweetSentiment
0,1232073974093352966,2020-02-24 22:45:00,I love to look at people's opinions about the ...,es,,,1
1,1223738854596927488,2020-02-01 22:44:13,The latest The AmericanAwaken ing Daily PM! T...,en,,,-1
2,1224564516748283904,2020-02-04 5:25:06,#Macau casinos will be closed for half a month...,en,,,-1
3,1232651956528799747,2020-02-26 13:01:42,Social media conspiracies blame coronavirus on...,ca,,,-1
4,1231469470020972545,2020-02-23 6:42:55,"Coronavirus, official: patient zero has nothin...",it,,,0


In [13]:
import pandas as pd
import numpy as np
!pip3 install tensorflow
!pip3 show tensorflow
!pip3 install keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.layers import Dropout
from keras.models import Sequential
from keras import layers
from keras import optimizers

Defaulting to user installation because normal site-packages is not writeable

ERROR: Exception:
Traceback (most recent call last):
  File "c:\program files\python37\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in _main
    status = self.run(options, args)
  File "c:\program files\python37\lib\site-packages\pip\_internal\cli\req_command.py", line 185, in wrapper
    return func(self, options, args)
  File "c:\program files\python37\lib\site-packages\pip\_internal\commands\install.py", line 333, in run
    reqs, check_supported_wheels=not options.target_dir
  File "c:\program files\python37\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 179, in resolve
    discovered_reqs.extend(self._resolve_one(requirement_set, req))
  File "c:\program files\python37\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 362, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "c:\program files\python37\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 314, in _get_abstra

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [9]:
X = df['tweetText'].values
X= X.astype(str)
y = df['tweetSentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X_train)

num_words=9000
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= num_words} # <= because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = num_words + 1

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 278
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath,encoding='utf-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix
    
embedding_dim = 300
embedding_matrix = create_embedding_matrix('glove.6B.300d.txt', tokenizer.word_index, embedding_dim)

nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
embedding_accuracy = nonzero_elements / vocab_size
print('embedding accuracy: ' + str(embedding_accuracy))


In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True))
model.add(layers.Conv1D(256, 3, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(28, activation='sigmoid'))
opt = optimizers.Adam(lr=0.0002)
model.compile(optimizer=opt, loss='binary_crossentropy')
model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
         ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]
res = model.fit(X_train, y_train, epochs=15, verbose=True, callbacks=callbacks, validation_data=(X_test, y_test), batch_size=100)

y_pred = model.predict(X_test)

In [ ]:
thresholds=[0.1,0.2,0.25,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Threshold: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(val, precision, recall, f1))

In [ ]:
column_names = ['pos', 'neg', 'neu']
f1_scores = []
threshold = 0.25
for i in range(0,28):
    emotion_prediction = y_pred[:,i]
    emotion_prediction[emotion_prediction>=threshold]=1
    emotion_prediction[emotion_prediction<threshold]=0
    emotion_test = y_test[:,i]
    precision = precision_score(emotion_test, emotion_prediction)
    recall = recall_score(emotion_test, emotion_prediction)
    f1 = f1_score(emotion_test, emotion_prediction)
    f1_scores.append(f1)
    print("Emotion: {}, Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(column_names[i], precision, recall, f1))

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.bar(column_names,f1_scores)
plt.xticks(rotation=90)
plt.show()